In [1]:
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import math
from keras.models import model_from_json
import os
from glob import glob
from scipy import stats as s
from sklearn.metrics import accuracy_score

# opening and store file in a variable

json_file = open("D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\modelsum2.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# use Keras model_from_json to make a loaded model

model = model_from_json(loaded_model_json)
print("model loaded")
# load weights into new model

model.load_weights("D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\FTry2.h5")
print("Loaded Model from disk")
print("weights loaded")
# compile and evaluate loaded model

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()
print("model prepared")

# getting the test list
f = open("D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\ucfTrainTestlist\\testlist0203.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

# creating the tags
train = pd.read_csv('D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\train_new4.csv')
y = train['class']
y = pd.get_dummies(y)
print("done")

# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    dhgy = "D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\UCF-103\\" + str(videoFile.split('/')[0]) + "/" + str(videoFile.split(' ')[0].split('/')[1])
    cap = cv2.VideoCapture(dhgy)  # capturing the video from the given path
    frameRate = cap.get(5)  # frame rate
    x = 1
    # removing all other files from the temp folder
    files3 = glob('D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\temp\\temp_2\\*')
    for f3 in files3:
        os.remove(f3)
    files4 = glob('D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\temp\\temp_1\\*')
    for f4 in files4:
        os.remove(f4)
    while cap.isOpened():
        frameId = cap.get(1)  # current frame number
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % math.floor(frameRate) == 0:
            # storing the frames in a new folder named train_1
            filename = 'D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\temp\\temp_2\\' + "_frame%d.jpg" % count
            count += 1
            cv2.imwrite(filename, frame)
            import cv2
            import numpy as np
            from PIL import Image
            from PIL import ImageFilter

            img = cv2.imread(filename)
            gamma = 1.1
            gamma_corrected = np.array(255 * (img / 255) ** gamma, dtype='uint8')
            img = cv2.cvtColor(gamma_corrected, cv2.COLOR_BGR2RGB)
            imageObject = Image.fromarray(img)
            sharpened1 = imageObject.filter(ImageFilter.SHARPEN)
            sharpened2 = sharpened1.filter(ImageFilter.SHARPEN)
            im2 = sharpened2.filter(ImageFilter.MedianFilter(size=3))
            filename1 = 'D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\temp\\temp_1\\' + "_frame%d.jpg" % count
            count += 1
            im2 = im2.save(filename1)
    cap.release()

    # reading all the frames from temp folder
    images = glob("D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\temp\\temp_1\\*.jpg")

    prediction_images = []
    for j in range(len(images)):
        img = image.load_img(images[j], target_size=(224, 224, 3))
        img = image.img_to_array(img)
        img = img / 255
        prediction_images.append(img)

    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])
    files1 = glob('D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\temp\\temp_1\\*')
    for f1 in files1:
        os.remove(f1)
    files2 = glob('D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\temp\\temp_2\\*')
    for f2 in files2:
        os.remove(f2)
    print(predict)
    print(actual)
# checking the accuracy of the predicted tags
accuracy_score(predict, actual) * 100


model loaded


  0%|                                                                                          | 0/208 [00:00<?, ?it/s]

Loaded Model from disk
weights loaded
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               25690368  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)   

  0%|▍                                                                                 | 1/208 [00:02<09:46,  2.83s/it]

['BabyCrawling']
['BabyCrawling']


...

100%|███████████████████████████████████████████████████████████████████████████████▌| 207/208 [05:28<00:01,  1.38s/it]

['BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Typing', 'Typing', 'Typing', 'Typing', 'Typing', 'Typing', 'Typing', 'Typin

100%|████████████████████████████████████████████████████████████████████████████████| 208/208 [05:29<00:00,  1.58s/it]

['BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'BabyCrawling', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Typing', 'Typing', 'Typing', 'Typing', 'Typing', 'Typing', 'Typing', 'Typin

97.11538461538461

In [2]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predict, actual, labels=['BabyCrawling','Typing','Surfing','WalkingWithDog','PlayingViolin','Knitting'])

array([[35,  1,  0,  0,  0,  0],
       [ 0, 39,  0,  0,  0,  0],
       [ 0,  0, 33,  1,  0,  0],
       [ 0,  0,  0, 35,  1,  0],
       [ 0,  0,  0,  0, 26,  0],
       [ 0,  3,  0,  0,  0, 34]], dtype=int64)

In [3]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(predict, actual, average='macro')

(0.9736936548952052, 0.9723252664429135, 0.97221840143095, None)

In [4]:
precision_recall_fscore_support(predict, actual, average='micro')

(0.9711538461538461, 0.9711538461538461, 0.9711538461538461, None)

In [5]:
precision_recall_fscore_support(predict, actual, average='weighted')

(0.9731208175975617, 0.9711538461538461, 0.9712937967119974, None)

In [6]:
precision_recall_fscore_support(predict, actual, average=None)

(array([1.        , 1.        , 0.96296296, 1.        , 0.90697674,
        0.97222222]),
 array([0.97222222, 0.91891892, 1.        , 0.97058824, 1.        ,
        0.97222222]),
 array([0.98591549, 0.95774648, 0.98113208, 0.98507463, 0.95121951,
        0.97222222]),
 array([36, 37, 26, 34, 39, 36], dtype=int64))

In [7]:
from sklearn.metrics import classification_report
print(classification_report(predict, actual, target_names=['BabyCrawling','Typing','Surfing','WalkingWithDog','PlayingViolin','Knitting']))

                precision    recall  f1-score   support

  BabyCrawling       1.00      0.97      0.99        36
        Typing       1.00      0.92      0.96        37
       Surfing       0.96      1.00      0.98        26
WalkingWithDog       1.00      0.97      0.99        34
 PlayingViolin       0.91      1.00      0.95        39
      Knitting       0.97      0.97      0.97        36

      accuracy                           0.97       208
     macro avg       0.97      0.97      0.97       208
  weighted avg       0.97      0.97      0.97       208



In [8]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(predict, actual, labels=['BabyCrawling','Typing','Surfing','WalkingWithDog','PlayingViolin','Knitting'])

array([[[172,   0],
        [  1,  35]],

       [[165,   4],
        [  0,  39]],

       [[174,   0],
        [  1,  33]],

       [[171,   1],
        [  1,  35]],

       [[181,   1],
        [  0,  26]],

       [[171,   0],
        [  3,  34]]], dtype=int64)